# Face-Mask-Classification Project


Authors:
+ Tobias Palmowski
+ Fabian Metz
+ Thilo Sander

Date of Midterm-Report: 29.03.2021 <br>
Date of final submission: 26.04.2021


### Introduction

This Jupyter Notebook is the core of the Face-Mask-Classification Project performed in the class "Machine Learning" of the Hertie School in Berlin. There is one other Jupyter Notebook which deals with combining the different datasets into one large data set - a task only performed once and therefore outsourced to another file.


### Data Processing: Pipeline-Building

<br>
<br>
<br>
<br>
[Short Description]

In [1]:
# Code for Pipeline-Building

### Baseline: SGD Classifier

<br>
<br>
<br>
<br>
[Short Description]

In [1]:
# Code for Baseline

### Evaluation of Baseline

<br>
<br>
<br>
<br>
[Short Description]

In [2]:
# Code for Baseline Evaluation